# **02. Computing HDX deuterated fractions from MD simulations**

In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.stats import linregress

In [ ]:
# Matplotlib settings for plotting
plt.rc('lines', linewidth=3, markersize=4)
plt.rc('axes', labelweight='heavy', labelsize=22, titlesize=22) # Default fontsizes for printing
plt.rc('axes.spines', top=False, right=False) # Switch off top/right axes
plt.rc('legend', fontsize=16) # Default fontsizes for printing
plt.rc('xtick', labelsize=16) # Default fontsizes for printing
plt.rc('ytick', labelsize=16) # Default fontsizes for printing
plt.rc('figure', titlesize=22, titleweight='heavy') # Default fontsizes for printing

In this notebook we will summarize the use of the **calc_hdx.py** wrapper script to calculate HDX-MS data for BPTI from an ensemble of stuctures sampled by MD simulations. As a reminder, the key property we must estimate for each residue is the backbone amide protection factor, $P_i$:

Many empirical predictive models have been proposed to estimate protection factors from protein structural metrics., denoted $P_{i}$ in the equation below, from structural information.

\begin{equation*}
D_{i,t} = 1 - exp(\frac{-k_{int}}{P_{i}}t)
\end{equation*}

Within a folded protein, a number of structural and dynamical factors can contribute to protecting a backbone amide from deuterium exchange. Unsurprisingly, that means that a wide variety of empirical predictive models have been developed to estimate protection factors from protein structural metrics. **calc_hdx.py** currently allows users to calculate protection factors using two methods, either
- the [_BestVendruscolo_](https://www.sciencedirect.com/science/article/pii/S0006349514006857?via%3Dihub) method, or
- the [_PerssonHalle_](https://www.pnas.org/content/112/33/10383) method.

Within HDXer, each method is named for its original developers. Both methods have been applied for computational estimates of HDX-MS in the past, including for BPTI, which was used as target data in the development of both approaches. However, applications with the _BestVendruscolo_ method have been more widespread, thanks to the additional time it has spent in usage (its initial development was over 15 years ago). Therefore, we will only focus on how to apply the _BestVendruscolo_ method throughout the tutorials.

The _BestVendruscolo_ model calculates $ln{P_i}$ as an ensemble average property of an atomistic set of protein structures (in our case, MD simulations):

\begin{equation*}
lnP_{i} = <\beta_{C}N_{i}^{C} + \beta_{H}N_{i}^{H}>
\end{equation*}

Here, $N_{i}^{C}$ represents the number of heavy-atom contacts, and $N_{i}^{H}$ the number of hydrogen bonds, involving each amide $i$. The two $\beta$ values represent individual empirical scaling factors for $N_{i}^{C}$ and $N_{i}^{H}$, first derived by [fitting to a set of experimental HDX data obtained for 7 soluble proteins](https://www.sciencedirect.com/science/article/pii/S0969212605003916?via%3Dihub).

First, we will go over commands that can be used to run **calc_hdx.py** and calculate deuterated fractions from a set of BPTI MD trajectories that we provide. Then, we will make plots using the output files provided by this script, to visualize the computed HDX-MS data and compare it to the experimental data we prepared for BPTI in the first notebook.

**calc_hdx.py** can be run directly from the Unix command line, with options for the HDX-MS calculations specified as command line arguments. There are a lot of options, so help text to describe the command line arguments is accessible by running `calc_hdx.py -h`:

In [ ]:
!python $HDXER_PATH/HDXer/calc_hdx.py -h
#!python $HDXER_PATH\HDXer\calc_hdx.py -h    # Use this command instead if you're running the tutorials on Windows

Some of these inputs are required for all **calc_hdx.py** runs, whereas some are optional arguments that allow extra control over the calculation methodology, or the analysis. We'll give a brief explanation of each flag below.

### **Mandatory inputs**

These inputs are required for all **calc_hdx.py** analyses. If a default isn't defined below, you'll have to explicitly use the argument when running **calc_hdx.py**
- -t: trajectory/trajectories of structural coordinates for analysis. **HDXer** is based on the MDTraj package, so any [trajectory file format supported by MDTraj](https://mdtraj.org/1.9.4/load_functions.html) will be supported by **HDXer**
- -p: topology/parameter file. This should exactly describe the coordinates supplied to the '-t' flag, e.g. if trajectories have been stripped of solvent, so too should the topology. Any format [supported by MDTraj](https://mdtraj.org/1.9.4/api/generated/mdtraj.Topology.html) will be read, but a PDB file will be sufficient.
- -m: the HDX predictive model (_BestVendruscolo_ or _PerssonHalle_). If the flag isn't provided, the analysis will default to the _BestVendruscolo_ method.
- -seg: a file containing the peptide segments for calculation of segment-averaged deuterated fractions. If the flag isn't provided, the analysis will search for a file named 'segfile.txt', and fail if the file cannot be found.

### **Optional inputs** 

These inputs are not required for a **calc_hdx.py** analysis, but you can use them to customize the analysis in a variety of ways
- -s, -e, -str: start, end, and stride frame numbers at which to read the input trajectories. HDXer assumes trajectories are independent replicates, and so if you provide multiple trajectories, each one will be truncated by the start, end, and stride values. By default, all the frames are read.
- -c: read the trajectories and perform the analysis in chunks of size '-c'. This can save memory if your trajectory is large and cannot be entirely read into memory at once.
- -sel: selection string [in MDTraj format](https://mdtraj.org/1.9.4/atom_selection.html) for the atoms to select for analysis from the trajectories. Useful if you want to include or exclude parts of the trajectory (e.g. ligands or lipids) from being included in the calculation of protection factors. See also the 'protonly' option accessible _via_ the '-mopt' flag.
- -dt: labeling times at which to calculate HDX-MS deuterated fractions
- -log: logfile to print out information messages and warnings
- -exp: file of experimental HDX-MS data for comparison to the computed values
- -out: prefix for output files resulting from the analysis
- -mopt: additional options to control the calculation _method_. A wide range of parameters can be defined for each empirical predictive model, details of the parameter names and values can be accessed as the `HDXer.methods.BV.params` attribute for the _BestVendruscolo_ method, or the `HDXer.methods.PH.params` attribute for the _PerssonHalle_ method. We'll give some examples of options you might like to control below.
- -aopt: additional options for the calculation _analysis_. By default, the HDXer analysis steps, which compare and plot the calculated data against experiment, use the same options as provided to the calculation _method_. However, options for analysis can also be independently controlled. This can be useful if, e.g., you want to re-run an identical HDX-MS calculation compared to a different set of experimental data. Parameters are accessible as the `HDXer.analysis.Analyze.params` attribute.

### **Running calc_hdx.py to calculate HDX-MS data**

In this section, we'll show you how to calculate HDX-MS data for an ensemble of BPTI structures that we have generated from our own BPTI simulations. Initiating from the [5PTI crystal structure](https://www.rcsb.org/structure/5PTI), we performed five replicate simulations of BPTI in a cubic box of TIP3P water. Following an initial equilibration period, each replicate simulation we provide consists of 500 ns of BPTI dynamics. We've made the trajectory files themselves (stripped of all water molecules to reduce the filesize) available at [this Zenodo data repository](https://dx.doi.org/10.5281/zenodo.4640761), licensed under a CC-BY license.

There are two options to carry out the next tutorial steps:

1. Download the trajectories we provide, and use the example script we provide to run **calc_hdx.py** on your local computer. The trajectory files are approximately 200 MB in size, and we expect the **calc_hdx.py** analysis would require approximately 1GB of RAM and complete in approximately 5-10 mins on a modern laptop or desktop
2. If you don't have the space or time to run your own estimates, we have packaged the output files from **calc_hdx.py** calculations inside the HDXer tutorial directories, so you can carry on directly with the notebook steps below. The results we have packaged should be identical to the results from running your own calculations


If you'd like to generate your own data, here is the script, *run_hdx.sh*, that you can use to calculate HDX-MS data for the BPTI MD trajectories.

```bash
#!/bin/bash
# A Bash script to run calc_hdx.py

python $HDXER_PATH/HDXer/calc_hdx.py -t $HDXER_PATH/tutorials/BPTI/BPTI_simulations/Run_1/bpti_5pti_reimg_protonly.xtc $HDXER_PATH/tutorials/BPTI/BPTI_simulations/Run_2/bpti_5pti_reimg_protonly.xtc $HDXER_PATH/tutorials/BPTI/BPTI_simulations/Run_3/bpti_5pti_reimg_protonly.xtc $HDXER_PATH/tutorials/BPTI/BPTI_simulations/Run_4/bpti_5pti_reimg_protonly.xtc $HDXER_PATH/tutorials/BPTI/BPTI_simulations/Run_5/bpti_5pti_reimg_protonly.xtc -p $HDXER_PATH/tutorials/BPTI/BPTI_simulations/Run_1/bpti_5pti_eq6_protonly.gro -m BestVendruscolo -log BPTI_calc_hdx.log -out BPTI_ -exp $HDXER_PATH/tutorials/BPTI/BPTI_expt_data/BPTI_expt_dfracs.dat -seg $HDXER_PATH/tutorials/BPTI/BPTI_expt_data/BPTI_residue_segs.txt -mopt "{ 'save_detailed' : True }"
```

As you can see, we provided some optional arguments to **calc_hdx.py**, as well as the mandatory inputs described above. We'll explain each argument in turn...

First, we provide the paths to the BPTI trajectories with the `-t` flag. Note that we provide the locations of all five trajectories at the same time: **calc_hdx.py** will concatenate the trajectories together for us. Next, we provide the topology of the BPTI protein to the `-p` flag. Note that the topology must be identical for *all* of the provided trajectories, i.e., in this case, a topology of the BPTI protein only, stripped of any surrounding solvent molecules. Then, we select the `BestVendruscolo` HDX predictive model, and define flags to create an output logfile called `BPTI_calc_hdx.log` and to create all other output files with a prefix of `BPTI_`. 

The `-exp` and `-seg` flags mean that successfully running **calc_hdx.py** will create files with estimated HDX deuterated fractions for the peptide segments provided in ```$HDXER_PATH/tutorials/BPTI/BPTI_expt_data/BPTI_residue_segs.txt```, and results will be compared to the experimental values in ```$HDXER_PATH/tutorials/BPTI/BPTI_expt_data/BPTI_expt_dfracs.dat```, which corresponds to the experimental BPTI data created in notebook 1. Exchange will be calculated for the default labeling times (experimental timepoints) of 0.167, 1.0, 10.0 and 120.0 minutes. Finally, the ```-mopt "{ 'save_detailed' : True }"``` flag saves output files containing the contacts and H-bonds calculated for each backbone amide at each frame of the trajectory. These files are not required for analysis with the **calc_hdx.py** script, but will be used for reweighting in later notebooks. Make sure to include this option if you want to perform reweighting with your own trajectories.

As outputs, **calc_hdx.py** will create files detailing the estimated protection factors and calculated intrinsic rates for each residue, the latter of which was actually used in notebook 1 to convert the experimentally-measured protection factors into fractional deuteration data. In addition, files containing the residue-level and peptide-segment-level fractional deuteration will be created, along with a number of plots to display the computed data in visual form in comparison to the provided experimental dataset. Finally, **calc_hdx.py** will also create one or more restart files, with a `.pkl` suffix, that can be used to re-run the analysis with alternative options more quickly (without having to read and store the trajectories in memory). As the `.pkl` files contain all the data structures used in a **calc_hdx.py** HDX-MS calculation, they can be quite large, so we suggest deleting the `*.pkl` files at the end of your project.

### **Customizing options for calc_hdx.py calculations**

As we mentioned already, **calc_hdx.py** offers a lot of customizable options for generating HDX-MS data. Some of these options can be used to adapt the model used to estimate protection factors (here the `BestVendruscolo` method), and some can be used to refine the calculation of residue intrinsic exchange rates to better suit particular experimental conditions. All of these method options are customizable using the `-mopt` flag to **calc_hdx.py**. Arguments to `-mopt` should be provided in the syntax of a Python dictionary, and surrounded by "double quotes" to make sure they're read correctly as a single argument. This might sound complex - so here are some example `-mopt` arguments: 

```bash
# This example will modify the estimation of protection factors using the BestVendruscolo method
# It will increase the distance cutoffs used to count hydrogen bonds and heavy-atom contacts between the backbone amides and their surroundings, to 0.26 nm and 0.7 nm respectively
# The default cutoffs are 0.24 nm radius from the amide hydrogen for H-bonds, and 0.65 nm from the amide nitrogen for heavy-atom contacts
-mopt "{ 'cut_Nh' : 0.26, 'cut_Nc' : 0.70 }"


# This example will modify the estimation of protection factors using the BestVendruscolo method
# It will include atoms from a ligand residue called 'LIG' in the calculation of hydrogen bonds and heavy-atom contacts for each protein backbone amide. Note, this also need us to provide an argument to the -sel flag!
# By default, only protein atoms are included in the calculations of H-bonds and contacts
-mopt "{ 'protonly' : False }" -sel "protein or resname LIG"


# This example will modify the estimation of protection factors using the BestVendruscolo method
# It will change the scaling parameters used in the BestVendruscolo model to estimate protection factors, beta_C and beta_H, to 0.25 and 3.0 respectively
# The default beta_C scaling parameter is 0.35, and the default beta_H parameter is 2.0
-mopt "{ 'betac' : 0.25, 'betah' : 2.0 }"


# This example will modify the calculation of residue intrinsic exchange rates
# It will change the experimental conditions of the exchange measurements to pD 6.4 and 310 K.
# The default conditions are pD 7.4 and 298 K
-mopt "{ 'kint_params' : { 'pD' : 6.4, 'Texp' : 310 } }"

```

If you have time, why not try out some of these options and see how they affect the estimated protection factors and deuterium exchange for the BPTI simulations? 

N.B. These are not the only method options that can be defined using the `-mopt` flag, although they do cover the most common options that you might like to vary during your own analyses. You can explore the full range of method options in the help text of the **HDXer** protection factor calculation classes. Parameters common to all predictive models can be seen _via_:
```
from HDXer.dfpred import DfPredictor
help(DfPredictor)
```
while options specific to the `BestVendruscolo` method can be seen with:
```
from HDXer.methods import BV
help(BV)
```

### **Visualizing calculated HDX-MS data**

Let's plot the estimated HDX deuterated fractions alongside the experimental HDX deuterated fractions to see how they differ.

The functions we use below are the same as the ones used in the previous notebook.

In [ ]:
times = np.array([0.167, 1.0, 10.0, 120.0]) # Create a NumPy array with time points in minutes

In [ ]:
def read_to_df(file):
    """Read and create a pandas DataFrame for the given argument.
    
    Args:
        file: There are four possible options:
            'segs' - peptide segments
            'expt' - experimental HDX deuterated fractions
            'pred' - calculated HDX deuterated fractions
            'reweighted' - reweighted HDX deuterated fractions
    
    Returns:
        df: A pandas DataFrame containing data for the given argument.
    """
    if file == 'segs':
        # Read and create a pandas DataFrame using a residue segments file
        df = pd.read_csv(os.path.expandvars('$HDXER_PATH/tutorials/BPTI/BPTI_expt_data/BPTI_residue_segs.txt'),
                         sep='\s+', header=None, names=['ResStr', 'ResEnd'])
    elif file == 'expt':
        # Read and create a pandas DataFrame using an experimental deuterated fractions file
        df = pd.read_csv(os.path.expandvars('$HDXER_PATH/tutorials/BPTI/BPTI_expt_data/BPTI_expt_dfracs.dat'), 
                         sep='\s+', skiprows=[0], header=None, usecols=[2, 3, 4, 5], names=times)
    elif file == 'pred':
        # Read and create a pandas DataFrame using a computed deuterated fractions file
        df = pd.read_csv(os.path.expandvars('$HDXER_PATH/tutorials/BPTI/BPTI_calc_hdx/BPTI_SUMMARY_segment_average_fractions.dat'), 
                         sep='\s+', skiprows=[0], header=None, usecols=[2, 3, 4, 5], names=times)
    elif file == 'reweighted':
        df = pd.read_csv(os.path.expandvars('$HDXER_PATH/tutorials/BPTI/BPTI_reweighting/reweighting_gamma_2x10^0_final_segment_fractions.dat'), 
                         sep='\s+', skiprows=[0], header=None, names=times)
    else:
        print("Incorrect argument given. Please choose one of the following: 'segs' 'expt' 'pred' 'reweighted'")
    return df

In [ ]:
def plot_dfracs(*args):
    """Plot HDX deuterated fractions for each time point.
    
    Args:
        *args: 
            'expt' - experimental HDX deuterated fractions
            'pred' - computed HDX deuterated fractions
            'reweighted' - reweighted HDX deuterated fractions
    """
    fig, axs = plt.subplots(len(times), 1, figsize=(12, 24))
    for i, (ax, t) in enumerate(zip(axs, times)):
        for arg in args:
            if arg in ['expt', 'pred', 'reweighted']:
                xs = np.arange(0, read_to_df(arg).iloc[:, 1].shape[0])
                ax.plot(xs, read_to_df(arg).iloc[:, i], label=arg)
                ax.set_title(f'Labeling time = {t} min')
                ax.legend(loc='upper right')
                ax.set_xticks(xs)
                ax.set_xlim(xs[0], xs[-1])
                ax.set_xticklabels(read_to_df('segs').iloc[:, 1], rotation=90)
                ax.set_ylim(0, 1)
            else:
                print("Incorrect argument given. Please choose one or more of the following: 'expt' 'pred' 'reweighted'")
    fig.text(0.5, 0.095, 'Residue', ha='center', fontsize=22)
    fig.text(0.05, 0.5, 'HDX deuterated fractions', va='center', rotation='vertical', fontsize=22)

In [ ]:
plot_dfracs('expt', 'pred')

To quantify the accuracy of our estimates, we can also calculate some simple descriptive statistics, such as the correlation and root-mean-square error (in units of fractional deuteration) between the computed and experimental data. We'll combine data for all of the residues and all of the labeling times together, to calculate the *overall* correlation between the calculated values and experiment, rather than correlation on a labeling-time by labeling-time basis:

In [ ]:
# Read in the experimental and calculated HDX-MS data
expt_data = read_to_df('expt')
pred_data = read_to_df('pred')

# Flatten the data arrays to combine all the labeling times together in a single dimension
expt_data_alltimes = expt_data.values.flatten()
pred_data_alltimes = pred_data.values.flatten()

# Fit a linear regression, and calculate the RMSE between computed and experimental data
slope, intercept, r, p, stderr = linregress(expt_data_alltimes, pred_data_alltimes)
rmse = np.sqrt(np.mean( (pred_data_alltimes - expt_data_alltimes)**2 ))

print("The R-squared between computed and experimental HDX-MS is: %3.2f" % r**2)
print("The linear regression between computed and experimental HDX-MS is: y = %3.2fx + %3.2f" % (slope, intercept))
print("The RMSE between computed and experimental HDX-MS is: %3.2f" % rmse)

A calculated $R^2$ of 0.59 suggests the experimental and calculated deuterated fractions correlate reasonably well, and from visual inspection the estimates do reflect the trends in deuteration across different protein regions well. However, there are clearly some mismatches, for individual residues or individual labeling times, between the experimental and the computed deuteration. Why might this be the case? It's difficult to know immediately why these inaccuracies might arise, and there may be many possible causes:
1. Uncertainty in the experimental HDX data (e.g. Perhaps there are measurement errors?)
2. Inaccuracy of the HDX predictive (forward) model (e.g. Perhaps the `PerssonHalle` model would be a more accurate estimator of protection factors in BPTI?)
3. Inadequate sampling of the structural ensemble (e.g. Perhaps the MD simulations are too short to capture all the relevant protein dynamics?)

Finally, related to the possibility of inadequate MD sampling, is the possibility that we have sampled the correct (relevant) *structures* in the MD simulations but just not in the correct relative *populations*.

To check for this possibility, we need a robust analysis method to obtain a structural ensemble that best-fits the experimental data, taking into account all possible forms of uncertainty. This is the concept of ensemble reweighting.

Conveniently, as we'll discuss, the objective HDXer reweighting process will also provide us with an opportunity to disentangle the effects of all of the different potential sources of error, and how they may contribute to the differences between our predictions and the experimental data.

We will use the `*.tmp` files created by **calc_hdx.py**, which include information about the number of contacts and hydrogen bonds calculated for each amide, to carry out ensemble reweighting of the BPTI traejctory frames in the next notebook *03_reweighting.ipynb*.